### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

In [3]:
pkl_directory = '../data/company_daily_price_pkl'
pkl_directory = './'
pkl_directory = '../data/company_pkl/'

In [4]:
hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol']
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [5]:
# investor.pkl, historical.pkl. 읽기
for key, val in code.items():
    pkl_name= '{}.pkl'.format(val[1])
    globals()['df_{}'.format(val[1])] = pd.read_pickle('./' + pkl_name)
    globals()['df_{}'.format(val[1])]['date'] = pd.to_datetime(globals()['df_{}'.format(val[1])]['date']) #change to timestamp
    globals()['df_{}'.format(val[1])]['date'] = globals()['df_{}'.format(val[1])]['date'].dt.date  # change to datetime
    pkl_name= '{}_historical.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'historical')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'historical')] = globals()['df_{}_{}'.format(val[1], 'historical')][hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생)
    globals()['df_{}_{}'.format(val[1], 'historical')]['date'] = globals()['df_{}_{}'.format(val[1], 'historical')]['date'].dt.date # change to datetime
    pkl_name= '{}_investors.pkl'.format(val[1])
    globals()['df_{}_{}'.format(val[1], 'investors')] = pd.read_pickle(pkl_directory + pkl_name)
    globals()['df_{}_{}'.format(val[1], 'investors')]['date'] = globals()['df_{}_{}'.format(val[1], 'investors')]['date'].dt.date # change to datetime

In [6]:
# index column : date
def find_previous_date(df_inv, df_his, date_current):
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)
        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:
            break
    return date2

In [7]:
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [8]:
# date column을 index로 옮김
df_inv = df_sec_investors.set_index('date')
df_his = df_sec_historical.set_index('date')
df_o = df_sec.set_index('date')

In [9]:
type(df_o.index[0]), type(df_inv.index[0]), type(df_his.index[0])

(datetime.date, datetime.date, datetime.date)

In [90]:
investor_rate = pd.DataFrame()
historical_rate = pd.DataFrame()
date_weekday = pd.DataFrame()
for date_current in df_o.index:
    date_previous = find_previous_date(df_inv, df_his, date_current)
    # 거래 전날 요일 구하기
    date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
    df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
    date_weekday = pd.concat([date_weekday, df_temp], axis=0)
    date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
    # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
    df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
    df_his_comp = df_his.loc[[date_previous_1, date_previous]]
    investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
    historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)

In [92]:
total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
total

,retail,foreigner,institution,financial,invtrust,pension,privequity,bank,insurance,financeetc,corporateetc,foreigneretc,open,high,low,close,vol,weekday,value
2022-06-20,8.195577,11.920678,-3.647354,7.363196,-30.591457,0.344811,23.893570,-2.813541,6.893887,-1.491670,1.854798,0.914929,-0.030995,-0.030744,-0.018182,-0.018062,0.210613,4,2
2022-06-21,-0.674957,-0.407361,2.593047,0.295068,-0.938927,-0.883900,-0.821502,-0.937576,-0.955874,0.062932,-0.814356,-0.661150,0.006734,0.0,-0.021886,-0.018395,0.152747,0,3
2022-06-22,-0.226413,-0.166064,-0.135501,0.034853,3.990909,4.034743,-0.044910,1.406863,4.635762,-0.988289,1.836559,-1.530848,-0.018395,-0.011686,0.001721,-0.003407,-0.261422,1,2
2022-06-23,-0.115765,-0.743679,-1.316192,-1.102356,-0.227930,0.236304,-4.804075,-1.851324,-0.502350,-21.944444,-0.254890,-1.760291,0.005111,-0.001689,-0.010309,-0.015385,-0.077913,2,2
2022-06-24,-1.523278,3.064922,-8.766880,-20.769609,-2.370714,-0.040771,-0.727922,2.363636,-5.573790,2.116711,-2.381156,-1.818471,-0.022034,-0.018613,-0.013889,-0.003472,0.196271,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-06,-0.635452,-0.254523,0.645057,0.252732,3.124629,0.597621,0.821393,66.172414,-0.145471,-0.963989,-1.054321,-0.835429,0.011076,0.001565,0.00639,0.004724,-0.344014,4,2
2023-02-07,-3.234200,-1.582361,-0.141131,0.833977,-0.886691,-2.654234,-0.752389,1.544148,-1.088161,-0.269231,-2.209475,-7.215278,-0.017214,-0.015625,-0.022222,-0.034483,0.000573,0,2
2023-02-08,-0.875136,-1.248176,-0.422927,-0.533907,-0.642540,-1.424225,-0.885376,-1.037732,-1.509524,-1.684211,-1.557954,-1.039106,-0.014331,-0.007937,0.0,0.00487,-0.071571,1,3
2023-02-09,-5.506210,4.372431,-0.326232,-0.168002,0.358792,-1.294586,-2.716667,-13.898396,-2.752336,-19.230769,1.841929,9.057143,0.01454,0.0128,0.012987,0.019386,-0.142601,2,3
